In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

2023-10-26 10:19:56.159823: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-26 10:19:56.230259: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES']= '3'
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

In [4]:
train_data = np.load('CIFAKE_Train.npz')
test_data = np.load('CIFAKE_Test.npz')

In [5]:
# Extract features and labels from the loaded data
x_train, y_train = train_data['images'], train_data['labels']
x_test, y_test = test_data['images'], test_data['labels']

In [6]:
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

2023-10-26 10:20:01.164331: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 17690 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:1e:00.0, compute capability: 7.5


In [7]:
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [8]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
acc = model.fit(x_train,y_train,epochs=10,batch_size=32,validation_split=0.2)
acc

Epoch 1/10


2023-10-26 10:20:31.143065: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902
2023-10-26 10:20:31.285158: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-10-26 10:20:32.905620: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f3c1cee8020 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-26 10:20:32.905652: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): Quadro RTX 6000, Compute Capability 7.5
2023-10-26 10:20:32.911517: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-26 10:20:32.990393: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-10-26 10:20:33.071084: I ./tensorflow/compiler/jit/device_compiler.h:1

2500/2500 [==============================] - 330s 106ms/step - loss: 0.2379 - accuracy: 0.9098 - val_loss: 0.1859 - val_accuracy: 0.9275
Epoch 2/10
2500/2500 [==============================] - 258s 103ms/step - loss: 0.1694 - accuracy: 0.9357 - val_loss: 0.2370 - val_accuracy: 0.8989
Epoch 3/10
2500/2500 [==============================] - 249s 100ms/step - loss: 0.1884 - accuracy: 0.9291 - val_loss: 0.1933 - val_accuracy: 0.9214
Epoch 4/10
2500/2500 [==============================] - 243s 97ms/step - loss: 0.1385 - accuracy: 0.9481 - val_loss: 0.1908 - val_accuracy: 0.9257
Epoch 5/10
2500/2500 [==============================] - 253s 101ms/step - loss: 0.1244 - accuracy: 0.9538 - val_loss: 0.1429 - val_accuracy: 0.9439
Epoch 6/10
2500/2500 [==============================] - 255s 102ms/step - loss: 0.1337 - accuracy: 0.9509 - val_loss: 0.2370 - val_accuracy: 0.9022
Epoch 7/10
2500/2500 [==============================] - 262s 105ms/step - loss: 0.1172 - accuracy: 0.9571 - val_loss: 0.1241

In [10]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)

625/625 [==============================] - 16s 25ms/step - loss: 0.1047 - accuracy: 0.9621
